<a href="https://colab.research.google.com/github/zachhom/170-P3-Neural-Net/blob/Zach/Copy_of_submission_fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Writeup

For the writeup, please include the following information:


*   Describe the architecture of your neural network
*   Discuss how you tuned your network and why you think it's performance is reasonable for this task
*   Discuss whether or not you feel that this classifier is appropriate for the given task (check PA3 description)



# Writeup

### `Describe the architecture of your neural network`

Our architecture uses the sequentially-layered architecture that adds the (Convolution, Max Pooling, Dropout) as a C-MP combination and Dense layer after two loops of layer propagation from C-MP layers, we further add Dense Layer to the model before the output layer.

In total we have 2 convolutional layers, 2 maxpooling layers, 2 hidden layers (256 and 64), and 4 dropout layers. 
### `Discuss how you tuned your network and why you think it's performance is reasonable for this task`

The overall idea is to `increase accuracy value` while `decreasing value loss`. Based on the overall trend while epochs being run, we attempted to tune the basic primitives constants like `Epoch` and `Batch Size Train`.

From our understanding, we have the best result when we operate on epoch/batch_train_size with learning rate 0.01. From our understanding, each epoch trains 100 units from the training dataset. We thought this makes sence because we have 10 categories to disclose the output type. Thus we would be training every category at least 5 times in an epoch loop. 

We got an overall trend of value accuracy of about 92 % and the value lost was around 0.17~0.20. This is the best result out of our set of data values among these categories.


### `Discuss whether or not you feel that this classifier is appropriate for the given task (check PA3 description)`

The performance for our network was 92%. Given a large dataset such as fashionMNIST, an 8% failure rate is a reasonable classifier because that means we have a relatively high success rate. Given that this is a clothing identification network, we can accept an error rate such as 8%. However, if the task was different such as facial recognition capability for a counter terrorism agency, an 8% failure rate would be too high.


# Tensorflow

## 0. Imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

## 1. Set Hyperparameters

In [ ]:
# Tune these

n_epochs = 125
batch_size_train = 4500
batch_size_test = 10000
learning_rate = 0.01

## 2. Import Data

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size_train)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size_test)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/fashion_mnist/3.0.1.incompleteSLHGXG/fashion_mnist-train.tfrecord*...:   0%|  …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/fashion_mnist/3.0.1.incompleteSLHGXG/fashion_mnist-test.tfrecord*...:   0%|   …

Dataset fashion_mnist downloaded and prepared to ~/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


## 3. Define Model

In [ ]:
model = tf.keras.models.Sequential()

# Convolutional Layers
# model.add(tf.keras.layers.Conv2D(20, (2,2), padding='same', activation='relu', input_shape=(28,28,1)))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
# model.add(tf.keras.layers.Conv2D(40, (2,2), padding='same', activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3)) 

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))


model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))


model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Dense Layers
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(32, activation='relu'))
# model.add(tf.keras.layers.Dense(16, activation='relu')) #256
# model.add(tf.keras.layers.Dense(126, activation='relu'))
# model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 32)          0

Things to consider:

* dropout, convensation, things like that
* overtrain




## 4. Train Model

In [ ]:
model.fit(
    ds_train,
    epochs=n_epochs,
    validation_data=ds_test,
)


Epoch 1/125


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


14/14 [==============================] - 21s 305ms/step - loss: 1.4108 - sparse_categorical_accuracy: 0.4800 - val_loss: 0.6770 - val_sparse_categorical_accuracy: 0.7485
Epoch 2/125
14/14 [==============================] - 2s 132ms/step - loss: 0.6629 - sparse_categorical_accuracy: 0.7547 - val_loss: 0.5087 - val_sparse_categorical_accuracy: 0.8032
Epoch 3/125
14/14 [==============================] - 2s 134ms/step - loss: 0.5371 - sparse_categorical_accuracy: 0.8004 - val_loss: 0.4391 - val_sparse_categorical_accuracy: 0.8349
Epoch 4/125
14/14 [==============================] - 2s 132ms/step - loss: 0.4767 - sparse_categorical_accuracy: 0.8261 - val_loss: 0.3831 - val_sparse_categorical_accuracy: 0.8578
Epoch 5/125
14/14 [==============================] - 2s 132ms/step - loss: 0.4307 - sparse_categorical_accuracy: 0.8441 - val_loss: 0.3553 - val_sparse_categorical_accuracy: 0.8704
Epoch 6/125
14/14 [==============================] - 2s 133ms/step - loss: 0.4005 - sparse_categorical_acc